# MuonDataLib Tutorial 5: Peak Property Filtering and Discrimination


In the previous tutorials we used the sample logs and times to filter the event data. These filters removed events corresponding to a whole frame. It is also possible to discriminate/filter on individual events. These filters operate on the properties of the peaks that define the events, for example the peak amplitude.  

First lets create a histogram for some unfiltered data. 


In [ ]:
from MuonDataLib.data.loader.load_events import load_events
from MuonDataLib.plot.basic import Figure
import os
import numpy as np

file_name = 'HIFI00195790.nxs'
input_file = os.path.join('..', '..', '..', '..', 'test', 'data_files', file_name)
data = load_events(input_file, 64)

no_filter_hist, bins = data.histogram()
fig = Figure(y_label='Counts')
fig.plot_from_histogram(bins, no_filter_hist, [0])
fig.show()

## Inspecting a Peak Property

At present the only available peak property is "Amplitudes", which relates to the height of the peak in the trace from the detector. Being able to descriminate/filter the events so that they all have amplitudes above some value is important for instrument calibration. To help the user decide on the value a method has been added to create a histogram of the peak property. 

In [ ]:
hist, bins = data.get_peak_property_histogram('Amplitudes')
fig = Figure(y_label='Counts', x_label='Amplitude')
fig.plot_peak_property_histogram(hist, bins, 'unfiltered, ')
fig.show()

Notice that the plot was generated with the `plot_peak_property_histogram` method. The above plot shows a rapid decrease in the number of events as the size of the amplitude increases. Lets remove the steepest part of the curve by using the `keep_data_peak_property_above` method to filter out the data below 2600.

In [ ]:
data.keep_data_peak_property_above("Amplitudes", 2600)

Next lets look at the impact it has on the histograms 

In [ ]:
hist_amp_filter, bins = data.histogram()
fig = Figure(y_label='Counts')
fig.plot_from_histogram(bins, no_filter_hist, [0], 'unfiltered, ')
fig.plot_from_histogram(bins, hist_amp_filter, [0], 'filtered (amplitude), ')
fig.show()

We can see that the filters have removed some data as expected.

## Removing the Filter

The filter value can be changed by using the 'keep_data_peak_property` method with the new threshold value. Whereas, the `delete_data_peak_property_above` method will remove the filter, in reality this sets the threshold to 0.

In [ ]:
data.delete_data_peak_property_above("Amplitudes")

Fianlly lets check that the histograms have been updated. 

In [ ]:
hist_del_filter, bins = data.histogram()
fig = Figure(y_label='Counts')
fig.plot_from_histogram(bins, no_filter_hist, [0], 'unfiltered, ')
fig.plot_from_histogram(bins, hist_del_filter, [0], 'filter removed, ')
fig.show()

As expected this data is identical to the unfiltered data. 